In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt

from getData import *
from vggFace import vggFace

model = vggFace()


Using TensorFlow backend.


Constructing model


loading weights


weights loaded


In [5]:
# CLEAR TEST/TRAIN DATA
tr_imgs = 0
test_imgs = 0

In [2]:

tr_imgs, tr_labels, tr_ids = getTrainingDataReplay_w_id()
test_imgs, test_labels,test_ids = getTestDataReplay_w_id()

# tr_imgs, tr_labels = getTrainingDataOuluP1()
# test_imgs, test_labels = getTestDataOuluP1()

print(tr_imgs.shape, tr_labels.shape)
print(test_imgs.shape, test_labels.shape)

(2400, 224, 224, 3) (2400,)
(3200, 224, 224, 3) (3200,)


In [ ]:
# test_imgs, test_labels = getTestDataOuluP1()
# test_imgs, test_labels = getTestDataReplay()
# print(test_imgs.shape, test_labels.shape)



In [10]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
# pad_model = Model(inputs=model.layers[0].input, outputs=model.get_layer('last_feature_layer').output)
x = GlobalAveragePooling2D()(model.get_layer('last_feature_layer').output)
x = Dense(100,activation='relu')(x)
x = Dense(80,activation='relu')(x)
x = Dense(1,activation='tanh', name='pad')(x)

y = Flatten()(model.get_layer("fr_classif5").output)
y = Dense(15)(y)
y = Activation('softmax', name='fr')(y)

pad_model = Model(inputs=model.layers[0].input, outputs=[x,y])


# for layer in pad_model.layers:
#     print(layer.name)

pad_model.get_layer("fr_classif1").trainable = False
pad_model.get_layer("fr_classif2").trainable = False
pad_model.get_layer("fr_classif3").trainable = False
pad_model.get_layer("fr_classif4").trainable = False
pad_model.get_layer("fr_classif5").trainable = False



pad_model.summary()

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])
losses = {'pad' : 'mse', 'fr' : 'categorical_crossentropy'}
# lossWeights = {"pad": 1.0, "fr": 1.0}
# loss_weights=lossWeights
pad_model.compile(optimizer='adam', loss=losses, metrics=['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
zero_padding2d_1_input (InputLa (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 226, 226, 3)  0           zero_padding2d_1_input[0][0]     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 226, 226, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
history = pad_model.fit(tr_imgs, {"pad":tr_labels, "fr":tr_ids}, batch_size=16, epochs=2)
print(history.history)
pad_model.save("padModel3_Replay_28122018.h5")


Epoch 1/2


  16/2400 [..............................] - ETA: 1:01:44 - loss: 6.3932 - pad_loss: 1.0295 - fr_loss: 5.3637 - pad_acc: 0.3125 - fr_acc: 0.1250

KeyboardInterrupt: 

In [ ]:
# history = model.fit(tr_imgs, tr_labels, batch_size=1, epochs=2)
# print(history.history)
# model.save("MyVGGface.h5")

In [1]:
# from keras.models import load_model
# model = load_model("MyVGGface.h5")
# pad_model = load_model("padModel.h5")

Using TensorFlow backend.


In [42]:
score = pad_model.evaluate(test_imgs,{"pad":test_labels,"fr":test_ids},batch_size=16 )
print(score)

  16/3200 [..............................] - ETA: 14s

  32/3200 [..............................] - ETA: 15s

  48/3200 [..............................] - ETA: 14s

  64/3200 [..............................] - ETA: 14s

  80/3200 [..............................] - ETA: 14s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 14s 5ms/step


[0.2172800602152933, 0.910625]


In [8]:
scores = pad_model.predict(test_imgs,batch_size=16,verbose=1)

  16/3200 [..............................] - ETA: 29s

  32/3200 [..............................] - ETA: 22s

  48/3200 [..............................] - ETA: 19s

  64/3200 [..............................] - ETA: 18s

  80/3200 [..............................] - ETA: 17s

...

3136/3200 [============================>.] - ETA: 0s

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 15s 5ms/step


In [ ]:
np.savetxt("scores/scores_model3_Replay_28122018.txt",scores)
np.savetxt("scores/true_labels_model3_Replay_28122018.txt",test_labels) #TODO maybe save PAD and FR scores separatly

In [ ]:
print(scores)

[[-0.99516386]
 [-0.7635757 ]
 [-0.99999994]
 ...
 [ 0.99604744]
 [ 0.9993456 ]
 [ 0.99867606]]


In [11]:
from keras.utils import plot_model
plot_model(pad_model, to_file='model.png',show_shapes=True)